In [2]:
from gensim import corpora, models, similarities


In [3]:
documents = ["Human machine interface for lab abc computer applications",
"A survey of user opinion of computer system response time",
"The EPS user interface management system",
"System and human system engineering testing of EPS",
"Relation of user perceived response time to error measurement",
"The generation of random binary unordered trees",
"The intersection graph of paths in trees",
"Graph minors IV Widths of trees and well quasi ordering",
"Graph minors A survey"]

In [4]:
documents

['Human machine interface for lab abc computer applications',
 'A survey of user opinion of computer system response time',
 'The EPS user interface management system',
 'System and human system engineering testing of EPS',
 'Relation of user perceived response time to error measurement',
 'The generation of random binary unordered trees',
 'The intersection graph of paths in trees',
 'Graph minors IV Widths of trees and well quasi ordering',
 'Graph minors A survey']

In [5]:
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist] for document in documents]
texts

[['human', 'machine', 'interface', 'lab', 'abc', 'computer', 'applications'],
 ['survey', 'user', 'opinion', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'management', 'system'],
 ['system', 'human', 'system', 'engineering', 'testing', 'eps'],
 ['relation', 'user', 'perceived', 'response', 'time', 'error', 'measurement'],
 ['generation', 'random', 'binary', 'unordered', 'trees'],
 ['intersection', 'graph', 'paths', 'trees'],
 ['graph', 'minors', 'iv', 'widths', 'trees', 'well', 'quasi', 'ordering'],
 ['graph', 'minors', 'survey']]

In [6]:
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1]for text in texts]
print texts

[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]


In [7]:
dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/deerwester.dict')
print(dictionary)

Dictionary(12 unique tokens: [u'minors', u'graph', u'system', u'trees', u'eps']...)


In [8]:
print(dictionary.token2id)

{u'minors': 11, u'graph': 10, u'system': 6, u'trees': 9, u'eps': 8, u'computer': 1, u'survey': 5, u'user': 7, u'human': 2, u'time': 4, u'interface': 0, u'response': 3}


In [9]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

[(1, 1), (2, 1)]


In [10]:
corpus = [dictionary.doc2bow(text) for text in texts]
# corpora.MmCorpus.serialize('/tmp/deerwester.mm', corpus)
print(corpus)

[[(0, 1), (1, 1), (2, 1)], [(1, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(0, 1), (6, 1), (7, 1), (8, 1)], [(2, 1), (6, 2), (8, 1)], [(3, 1), (4, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(5, 1), (10, 1), (11, 1)]]


In [11]:
class MyCorpus(object):
    def __iter__(self):
        for line in open("/data/opinion_spam/real/10lines"):
            yield dictionary.doc2bow(line.lower().split())
corpus_mem = MyCorpus()
print corpus_mem

In [12]:
for v in corpus_mem:
    print v

[]
[(4, 2)]
[(4, 1)]
[(2, 1), (4, 1)]
[]
[]
[(4, 2)]
[(2, 1)]
[]
[(2, 1)]


In [13]:
dictionary = corpora.Dictionary(line.lower().split() for line in open('/data/opinion_spam/real/10lines'))
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.iteritems() if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)
dictionary.compactify()
print dictionary

Dictionary(289 unique tokens: [u'impression', u'walking', u'just', u'show', u'when']...)


In [17]:
#Test w2v
import gensim
sentences = [[word for word in line.split()]for line in open('/data/opinion_spam/real/10lines')]
print sentences[0]
model_10 = gensim.models.Word2Vec(sentences, min_count=5, size=100, window=5, workers=8)
print model_10
print type(model_10.vocab)
print model_10['film']

['A1004AX2J2HXGL', 'B0007RT9LC', 'May', '30,', '2005', '3', '4', '5.0', 'The', 'film', 'speaks', 'for', 'itself', 'The', 'only', 'thing', 'missing', 'is', 'a', 'presentation', 'of', 'the', 'original', '78', 'minute', 'film', '"Some', 'Folks', 'Call', 'it', 'a', 'Sling', 'Blade"', '(1993)', 'which', 'later', 'the', 'feature', 'length', 'film', 'was', 'created', 'from.', "Can't", 'have', 'it', 'all,', 'I', 'guess.', 'Perhaps', 'when', 'they', 'release', 'the', 'extra-special', 'edition?', 'If', 'you', 'like', 'Bill', 'Bob', 'in', 'this', 'film,', 'you', 'should', 'also', 'check', 'out', '"Dead', 'Man"', "(he's", 'in', 'exactly', 'one', 'hilarious', 'scene)', 'and', '"The', 'Man', 'Who', "Wasn't", 'There".', 'Approximate', 'Run', 'Time:', '148', 'minutes', 'Bonus', 'Feature(s):', '*', 'Feature', 'Commentary', 'With', 'Writer/Director/Actor', 'Billy', 'Bob', 'Thornton', '*', 'Mr.', 'Thornton', 'Goes', 'To', 'Hollywood', '*', 'Bravo', 'Profiles:', 'Billy', 'Bob', 'Thornton', '*', 'A', 'Roun

In [16]:
#Test w2v
import gensim
sentences = [[word for word in line.split()]for line in open('/data/opinion_spam/real/reviewsNew.txt')]
print sentences[0]
model = gensim.models.Word2Vec(sentences, min_count=5, size=100, window=5, workers=8)
print model
print type(model.vocab)


['A1004AX2J2HXGL', 'B0007RT9LC', 'May', '30,', '2005', '3', '4', '5.0', 'The', 'film', 'speaks', 'for', 'itself', 'The', 'only', 'thing', 'missing', 'is', 'a', 'presentation', 'of', 'the', 'original', '78', 'minute', 'film', '"Some', 'Folks', 'Call', 'it', 'a', 'Sling', 'Blade"', '(1993)', 'which', 'later', 'the', 'feature', 'length', 'film', 'was', 'created', 'from.', "Can't", 'have', 'it', 'all,', 'I', 'guess.', 'Perhaps', 'when', 'they', 'release', 'the', 'extra-special', 'edition?', 'If', 'you', 'like', 'Bill', 'Bob', 'in', 'this', 'film,', 'you', 'should', 'also', 'check', 'out', '"Dead', 'Man"', "(he's", 'in', 'exactly', 'one', 'hilarious', 'scene)', 'and', '"The', 'Man', 'Who', "Wasn't", 'There".', 'Approximate', 'Run', 'Time:', '148', 'minutes', 'Bonus', 'Feature(s):', '*', 'Feature', 'Commentary', 'With', 'Writer/Director/Actor', 'Billy', 'Bob', 'Thornton', '*', 'Mr.', 'Thornton', 'Goes', 'To', 'Hollywood', '*', 'Bravo', 'Profiles:', 'Billy', 'Bob', 'Thornton', '*', 'A', 'Roun

KeyboardInterrupt: 

In [ ]:
#for key in model.vocab:
#    print key, "===>", model.vocab.get(key)
#print model['cooking']
#print len(model['cooking'])
print model.similarity('computer', 'film')
print model.similarity('movie', 'film')
print model.similarity('book', 'film')

In [ ]:
#model.save('/data/opinion_spam/real/w2v.total')

In [1]:
print model.similarity('book', 'film')

NameError: name 'model' is not defined